# 東京都23区ごとの放置自転車数を解析しよう

### このノートブックでやること

* ArcGIS Online へ ArcGIS for Developers のアカウントを使ってログインする
* Jupyter Notebook へ地図を表示させる
* フィーチャ レイヤーの作成①：CSV の中身を pandas で確認しよう！
* フィーチャ レイヤーの作成②：CSV からフィーチャサービスを公開しよう！※フィーチャ サービスは作成したらどんなもんか見てみよう
* フィーチャ レイヤーの作成③：公開したフィーチャサービスを Jupyter Notebook の地図に追加しよう！
* 解析：東京都 23 区ごとの放置自転車数を解析する
* グラフ表示
* Web マップの形で出力しよう


### ArcGIS for Develoersのアカウントを使ってログインする

In [ ]:
# ArcGIS for Developers ログイン情報
develoersUser = ''
develoersPass = ''

In [ ]:
from arcgis.gis import GIS
gis = GIS("http://www.arcgis.com/",develoersUser,develoersPass)
user = gis.users.get(develoersUser)
user

In [ ]:
# ArcGIS for Developers で使えるクレジット数（おまけ）
beforeCredit = gis.admin.credits.credits
beforeCredit

### Jupyter Notebookへ地図を表示させる

In [ ]:
geodevmap = gis.map("東京都", zoomlevel = 12)
geodevmap

### フィーチャ レイヤーの作成①：CSVの中身をpandasで確認しよう！
手順
* (データ準備)
* データ読み込み
* 中身の確認

##### データ取得

使用する CSV ファイルを入手します。
githubからcloneまたはzipでノートブックをダウンロードすると次のフォルダに格納しています。

C:<任意のフォルダ>\arcgis-samples-python-api-master\arcgis-samples-python-api-master>tree /f
-  handson\meetup
         │  Meetup Hands On!.ipynb
         │
         └─data
             illegal_parking_2015.csv

ダウンロード元

[東京都内の放置自転車（2015）](https://hub.arcgis.com/datasets/ejopendataportal::%E6%9D%B1%E4%BA%AC%E9%83%BD%E5%86%85%E3%81%AE%E6%94%BE%E7%BD%AE%E8%87%AA%E8%BB%A2%E8%BB%8A%EF%BC%882015%EF%BC%89)

ページ右側の「ダウンロード」ボタンから[スプレッドシート]を選択して任意の場所に保存します。

ファイル名は次でお願いします
* illegal_parking_2015.csv


##### 中身の確認

In [ ]:
# csvファイルの読み込み
illegal_parking_csv = 'data/illegal_parking_2015.csv'
illegal_parking_csv

In [ ]:
import pandas as pd
from IPython.display import display
dataframe = pd.read_csv(illegal_parking_csv)
display(dataframe)

### フィーチャ レイヤーの作成②：CSVからフィーチャサービスを公開しよう！

In [ ]:
# CSV ファイルを ArcGIS Online へアップロード
illegal_parking_bicycles = gis.content.add({},illegal_parking_csv)

In [ ]:
# アップロードした CSV ファイルもとにフィーチャ レイヤーを作成・公開する
illegal_parking_bicycles_layer = illegal_parking_bicycles.publish()
illegal_parking_bicycles_layer
# ArcGIS Online へログインしてアイテムを確認してみましょう。

### フィーチャ レイヤーの作成③：公開したフィーチャサービスを Jupyter Notebook の地図に追加しよう！

In [ ]:
geodevmap.add_layer(illegal_parking_bicycles_layer)
# このあと地図をみてみましょう！↑

### 解析：東京都 23 区ごとの放置自転車数を解析する
東京都のポリゴン（図形データ）を使って、区ごとに次のことを解析して求めます。
* ポリゴン内のポイントの数
* ポリゴン内のそれぞれのポイントが持つ属性情報で「放置自転車数」を合算する

手順
* 東京都のポリゴンデータを呼び出す
* ArcGIS Online のエリア集計の解析を実行する


##### 東京都のポリゴンデータを呼び出す

In [ ]:
# 目的データの検索
flayer_search_result = gis.content.search("title:境界線表示用 AND owner:Chikako_OD","Feature Layer", outside_org=True)
flayer_search_result

# cf :ArcGIS Opendata 
# http://hub.arcgis.com/datasets/ejopendataportal::%E5%B9%B3%E6%88%90-24-%E5%B9%B4%E7%B5%8C%E6%B8%88%E3%82%BB%E3%83%B3%E3%82%B5%E3%82%B9-%E6%B4%BB%E5%8B%95%E8%AA%BF%E6%9F%BB%EF%BC%88%E6%9D%B1%E4%BA%AC%E9%83%BD-23-%E5%8C%BA%E3%80%81%E5%A2%83%E7%95%8C%E7%B7%9A%E8%A1%A8%E7%A4%BA%E7%94%A8%EF%BC%89

In [ ]:
# 東京都のポリゴンレイヤーを表示している地図に重ねる
geodevmap.add_layer(flayer_search_result[0])
# このあと地図をみてみましょう！↑

##### ArcGIS Online のエリア集計の解析を実行する

In [ ]:
from arcgis.features import summarize_data
summarize_layer = summarize_data.summarize_within(flayer_search_result[0],
                                illegal_parking_bicycles_layer,
                                sum_shape=True,
                                summary_fields=["放置台数_自転車 SUM"],
                                output_name='東京都エリア集計_2015',
                                gis=gis)
summarize_layer

In [ ]:
# 解析結果を地図へ表示する
geodevmap.add_layer(summarize_layer)
# このあと地図をみてみましょう！↑

### グラフ表示
エリア集計解析結果を見てみよう

In [ ]:
import requests
import io
import pandas
from IPython.display import display
import json

In [ ]:
layer = summarize_layer.layers[0]
layer.query().df

#### 必要な要素だけで見てみよう

In [ ]:
illegal_parking_bicycles_df = layer.query().df
# 3つの要素だけで表示
filtered_df = illegal_parking_bicycles_df[['CSS_NAME', 'SUM_放置台数_自転車','Point_Count']]
# 要素でソートして表示
sorted_df = filtered_df.sort_values(by='SUM_放置台数_自転車',ascending=False)
sorted_df

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
# matplotlibはデフォルトで日本語フォントをもっていないそうなので、豆腐（文字化け）の場合はフォントを指定します。
fp = FontProperties(fname=r'data/ipaexg.ttf', size=11)
# fp = FontProperties(fname=r'D:\\ProgramFiles\\Fonts\\ipaexg.ttf', size=11)
# グラフの設定
graphdata = sorted_df.plot(x='CSS_NAME', y='SUM_放置台数_自転車', kind='bar', figsize=(10,10))
graphdata.set_xticklabels(sorted_df['CSS_NAME'], fontproperties=fp)
graphdata.set_xlabel('CSS_NAME', fontproperties=fp)
plt.show()

### Web マップ作成

In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.mapping import WebMap

web_map_properties = {'title':'MeetupHandsOn228',
                     'snippet':'GeoDev Meetup ハンズオンで作成した Web マップ',
                     'tags':'geodev_jp'}

In [ ]:
web_map_item = geodevmap.save(item_properties=web_map_properties)
web_map_item

In [ ]:
# 指定のURLを別ブラウザで表示する
import webbrowser
url = "http://www.arcgis.com/home/webmap/viewer.html?webmap=" + web_map_item.itemid
webbrowser.open(url)